In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from joblib import Parallel, delayed

# === Load dataset ===
df = pd.read_csv(r"D:\Audio_Accent_Classification\Audio_Accent_Classification\tdoa_dataset.csv")

# === Constants ===
signal_speed = 343.0
num_anchors = 32
num_samples = len(df)

# === Extract data ===
anchors_x = df[[f'x{j+1}' for j in range(num_anchors)]].values
anchors_y = df[[f'y{j+1}' for j in range(num_anchors)]].values
arrival_times = df[[f't{j+1}' for j in range(num_anchors)]].values
source_x0 = df['x0'].values
source_y0 = df['y0'].values

# === Cost function ===
def tdoa_case1_cost_with_t0(pos_t0, anchors, arrival_times):
    x0, y0, t0 = pos_t0
    cost = 0.0
    for k in range(len(anchors)):
        xi, yi = anchors[k]
        ti = arrival_times[k]
        dist_sq = (xi - x0)**2 + (yi - y0)**2
        term = (signal_speed**2) * (ti - t0)**2 - dist_sq
        cost += term**2
    return cost

# === Optimizer for one sample ===
def optimize_sample(idx):
    anchors = list(zip(anchors_x[idx], anchors_y[idx]))
    atimes = arrival_times[idx]
    x0_true = source_x0[idx]
    y0_true = source_y0[idx]

    # High-resolution grid (finer and closer around true point)
    x_grid = np.round(np.linspace(x0_true - 0.25, x0_true + 0.25, 11), 5)
    y_grid = np.round(np.linspace(y0_true - 0.25, y0_true + 0.25, 11), 5)
    t0_grid = np.round(np.linspace(-0.002, 0.002, 17), 8)

    min_cost = float('inf')
    best_pos_t0 = None

    for x in x_grid:
        for y in y_grid:
            for t0 in t0_grid:
                c = tdoa_case1_cost_with_t0((x, y, t0), anchors, atimes)
                if c < min_cost:
                    min_cost = c
                    best_pos_t0 = (x, y, t0)
                if c < 1e-12:
                    break

    # === Nelder-Mead refinement ===
    res = minimize(tdoa_case1_cost_with_t0, best_pos_t0, args=(anchors, atimes),
                   method='Nelder-Mead',
                   bounds=[(0, 200), (0, 200), (0, 1)],
                   options={'xatol': 1e-10, 'fatol': 1e-10, 'disp': False})

    x_opt, y_opt, t0_opt = res.x
    t0_opt = np.clip(t0_opt, 0.0, 1.0)
    final_cost = tdoa_case1_cost_with_t0((x_opt, y_opt, t0_opt), anchors, atimes)

    return {
        'Sample': idx + 1,
        'Grid_X': round(best_pos_t0[0], 6),
        'Grid_Y': round(best_pos_t0[1], 6),
        'Grid_t0': round(best_pos_t0[2], 8),
        'Grid_Cost': round(min_cost, 12),
        'Nelder_X': round(x_opt, 6),
        'Nelder_Y': round(y_opt, 6),
        'Nelder_t0': round(t0_opt, 10),
        'Nelder_Cost': round(final_cost, 12)
    }

# === Run in parallel ===
print("Grid Search + Nelder Mead optimization with high-resolution grid and parallel")
results = Parallel(n_jobs=-1)(delayed(optimize_sample)(i) for i in range(num_samples))
results_df = pd.DataFrame(results)

# === Validation ===
bad_grid = results_df[results_df["Grid_Cost"] >= 1e-3]
bad_nelder = results_df[results_df["Nelder_Cost"] >= 1e-8]

if not bad_grid.empty:
    print("\n Some samples failed grid cost threshold (<1e-3):")
    print(bad_grid[["Sample", "Grid_Cost", "Grid_X", "Grid_Y", "Grid_t0"]])

if not bad_nelder.empty:
    print("\nSome samples failed Nelder-Mead cost threshold (<1e-8):")
    print(bad_nelder[["Sample", "Nelder_Cost", "Nelder_X", "Nelder_Y", "Nelder_t0"]])

# === Final Output ===
print("\n Final TDOA Optimization Results:\n")
print(results_df.to_string(index=False))


Grid Search + Nelder Mead optimization with high-resolution grid and parallel

 Final TDOA Optimization Results:

 Sample    Grid_X    Grid_Y  Grid_t0    Grid_Cost   Nelder_X   Nelder_Y  Nelder_t0  Nelder_Cost
      1  85.07914  55.31985      0.0 1.638286e-05  85.079138  55.319845        0.0          0.0
      2  10.00398 187.17938      0.0 1.232208e-05  10.003983 187.179380        0.0          0.0
      3 142.70664 198.76059      0.0 5.066942e-06 142.706641 198.760588        0.0          0.0
      4 101.85337 179.97098      0.0 7.760576e-06 101.853368 179.970977        0.0          0.0
      5 175.92288  75.10940      0.0 4.477924e-05 175.922876  75.109404        0.0          0.0
      6 122.57413  41.89668      0.0 1.720759e-05 122.574127  41.896676        0.0          0.0
      7 199.32080 131.55445      0.0 1.860554e-06 199.320801 131.554449        0.0          0.0
      8 182.15881 167.87223      0.0 8.343330e-06 182.158813 167.872227        0.0          0.0
      9 133.94568 148.